In [5]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy.io
import math
import matplotlib.pyplot as plt
import subprocess
import time
from PIL import Image

In [104]:
def thissoftmax(Z):
    
    #Zshift = Z - np.max(Z)
    
    t = np.exp(Z)
    a = np.divide(t, (np.sum(t, axis=0, keepdims=True)))
    
    return a, Z

In [162]:
def softmax_backward_extended(dA, dJdAMAtrix, cache, Y):
    """Unvectorized computation of the gradient of softmax.
 
    z: (T, 1) column array of input values.
 
    Returns D (T, T) the Jacobian matrix of softmax(z) at the given z. D[i, j]
    is DjSi - the partial derivative of Si w.r.t. input j.
    """
    Z = cache    
    a, _ = thissoftmax(Z)
    
    assert(Z.shape == dA.shape)
    
    noOfClass, m = Z.shape
    
    dJdZMatrix = np.zeros([noOfClass, 1])
    #print(dA[:,1])
    
    for k in range(m):
        
        dAdZ = np.zeros((noOfClass, noOfClass))
        
        for i in range(noOfClass):
            for j in range(noOfClass):
                dAdZ[i, j] = a[i, k] * ((i == j) - a[j, k])

        dAdZMatrix = -np.outer(a[:, k], a[:, k]) + np.diag(a[:, k].flatten())
        
        if (np.sum(dAdZ) - np.sum(dAdZMatrix)) > 1e-15:
            print("difference for dAdZ and Matrix is too big")
        assert(dAdZMatrix.shape ==  dAdZ.shape)
        
        new_vector = np.zeros((noOfClass, 1))
        
        #print((dA[:,k].reshape(noOfClass,1) * D).shape)
        """
        for j in range(noOfClass):
            if j ==0:
                temp_vector = np.sum (dJdAMAtrix[k,j, :,:] * dAdZMatrix, axis=1, keepdims=True)
            else:
                temp_vector = temp_vector + np.sum (dJdAMAtrix[k,j, :,:] * dAdZMatrix, axis=1, keepdims=True)
                
            new_vector = temp_vector
        #new_vector = np.sum (dJdAMAtrix[k,:,:] * dAdZMatrix, axis=1, keepdims=True)
        
        #print(dJdAMAtrix[k,:,:].shape)
        new_vector.reshape(noOfClass,1)
        """
        new_vector = np.sum ( (dA[:,k].reshape(noOfClass,1) * dAdZMatrix).T, axis=1, keepdims=True)
        #print(new_vector.shape)
        #new_vector.shape
        #print(dA[:,k].reshape(noOfClass,1).shape)
        #new1_vector = np.sum( dA[:,k].reshape(noOfClass,1) * dAdZ  , axis = 1, keepdims=True)
        #new_vector = dA[:,k].reshape(noOfClass,1) * np.sum(dAdZ, axis = 1, keepdims=True)
        
        #print(new_vector.shape)
        
        if k == 0:
            dJdZMatrix = new_vector

        else:
            dJdZMatrix = np.concatenate((dJdZMatrix, new_vector), axis=1)
            
            

    dJdZa = a + dA*a
    
    #print(dJdZa[:,1] - dJdZMatrix[:,1])
    
    return dJdZMatrix, dJdZa

In [163]:
m = 4000
noOfClass = 10

Z = np.random.rand(noOfClass,m)

Y = (Z == Z.max(axis=0)[None, :]).astype(int)
AL, Z = thissoftmax(Z)

### simple way ###
DJDAL = -1 * Y/AL
for i in range(m):
    if np.sum(DJDAL[:,i] == 0.0) != 9:
        print("error")

dALMatrix = np.zeros([m, noOfClass ,noOfClass, noOfClass])
for k in range(m):
    #print(1/AL[:,i].reshape(noOfClass,1).shape)
    tempMatrix = -1 * np.outer(Y[:,k].reshape(noOfClass,1), 1./(AL[:,k].reshape(noOfClass,1)))
    for l in range(noOfClass):
        tempMatrix1 = tempMatrix
        tempMatrix1[l,:] = 0.0
        tempMatrix1[:,l] = 0.0
        #print(tempMatrix1)
        dALMatrix[k,l,:,:] = tempMatrix1
    np.set_printoptions(linewidth=120) 
    #for i in range(noOfClass):
        #for j in range(noOfClass):
            #if i != j:
                #if dALMatrix[k,i,j] != 0.0:
                    #print(dALMatrix[k,i,j])
#print(dALMatrix[1,:,:])

#for i in range(m):
#    diff  = (np.sum(np.diag(dALMatrix[i,i,:,:]) - DJDAL[:,i]))
#    if diff > 1e-15:
#        print("diff between 2 methods is too big: " + str(diff) )



#print(DJDAL.shape)

dAns = AL - Y

dJdZMatrix, dJdZa = softmax_backward_extended(DJDAL, dALMatrix, Z, Y)
#print(dJdZa.shape)

In [164]:
print(np.sum(dJdZMatrix) - np.sum(dJdZa))
print(np.sum(dAns) - np.sum(dJdZMatrix))

-3.90798504668e-14
7.1054273576e-15


In [6]:
#D = -np.outer(Z, Z) + np.diag(Z.flatten())
#print(D)